In [2]:
import pandas as pd 
import os, fnmatch
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# import tflite_runtime.interpreter as tflite
# bts_df = pd.DataFrame()
# listOfFolders= os.listdir('./raw_data')
# print(listOfFolders)
# pattern = "*.csv"
# Check every folder in the list of folders



In [3]:
tflite

NameError: name 'tflite' is not defined

In [ ]:
# for folder in listOfFolders:
#     if not folder.startswith("."): 
#         listOfFiles = os.listdir('./raw_data/{}'.format(folder))

#         # Check everyfile in that folder
#         for files in listOfFiles:
#             try:
#                 if fnmatch.fnmatch(files, pattern):
#                     current_fn = "./raw_data/{}/{}".format(folder, files)
#                     cur_df = pd.read_csv(current_fn)
#                     bts_df = bts_df.append(cur_df)
#                     print("processing: {}".format(current_fn))
#                     bts_df = bts_df.dropna(subset=['reading_time'])
#                     bts_df["unix_timestamp"] = pd.to_datetime(bts_df["reading_time"]).astype(int)
#                     mean_time = bts_df["unix_timestamp"].mean()
#                     min_time = bts_df["unix_timestamp"].min()
#                     bts_df["norm_time"] = (bts_df["unix_timestamp"]-mean_time)/(3600*1000000000)
#                     bts_df = bts_df.sort_values(by=['norm_time'])
#                     bts_df.drop(["reading_time"], axis='columns', inplace=True)

#                     bts_df_grouped = bts_df.groupby(["station_id","parameter_id"])
#                     for key,item in bts_df_grouped:
#                         if key[1] == 122 and key[0] == 1161114004:
#                             sub_data = bts_df_grouped.get_group(key)
# #                             mean_val = sub_data['value'].mean()
# #                             sub_data['norm_value'] = sub_data['value']-mean_val
# #                             max_val = sub_data['norm_value'].max()
# #                             sub_data['norm_value'] = sub_data['norm_value']/max_val
#                             sub_data.sort_values(by=['norm_time']).to_csv("./grouped_data/{}_{}_{}_.csv".format(folder, key[0],key[1]), index=False)
#                             print("Finish: {}".format(key))
#             except Exception as e:
#                 print(e)

In [ ]:
daily_prediction_result = pd.read_csv("Result/12_05_21.csv", names=["id","station_id","parameter_id","unix_timestamp","norm_time","value", "label", "prediction"])



In [ ]:
daily_prediction_result

In [ ]:
y_true = np.array(daily_prediction_result["label"])
y_pred = np.array(daily_prediction_result["prediction"])

In [ ]:

# Mean Squared Error
MSE = np.square(np.subtract(y_true,y_pred)).mean()
MSE

In [14]:
    # Read dataset from file
    raw_dataset = pd.read_csv("/Users/nguyenlinh/Macadamia/Advanced_SS_project/MLService/Result/12_05_21.csv", 
        names=["id","station_id","parameter_id","unix_timestamp","norm_time","value", "label", "prediction"])
    mean_val = raw_dataset['value'].mean()
    raw_dataset['norm_value'] = raw_dataset['value']-mean_val
    max_val = raw_dataset['norm_value'].max()
    raw_dataset['norm_value'] = raw_dataset['norm_value']/max_val
    raw_dataset = raw_dataset.astype({'id':'float','value':'float', 'station_id':'int', 'parameter_id':'int', 'unix_timestamp':'int', 'label':'float', 'prediction':'float'})
    
    dataset = raw_dataset.copy()
    dataset = dataset.dropna().drop(['id','station_id','parameter_id','unix_timestamp', "label", "prediction"], axis=1)
    dataset_full = dataset.sort_values(by=['norm_time'])
    dataset = dataset_full[0:300]

    # pre-processing data 

    serial_data = dataset.drop(["value", "norm_time"], axis=1)
    serial_data['norm_1'] = serial_data['norm_value'].shift(1)
    serial_data['norm_2'] = serial_data['norm_value'].shift(2)
    serial_data['norm_3'] = serial_data['norm_value'].shift(3)
    serial_data['norm_4'] = serial_data['norm_value'].shift(4)
    serial_data['norm_5'] = serial_data['norm_value'].shift(5)
    serial_data['norm_6'] = serial_data['norm_value'].shift(6)
    serial_data = serial_data[6:]

   # Split data into training and testing
    train_dataset = serial_data.sample(frac=0.8, random_state=1)
    test_dataset = serial_data.drop(train_dataset.index)
    train_features = np.array(train_dataset.drop(['norm_value'], axis=1))
    train_features = np.array(train_features)[:,:,np.newaxis]
    train_labels = np.array(train_dataset.drop(['norm_6'], axis=1))
    train_labels = train_labels.reshape(train_labels.shape[0],train_labels.shape[1],1)
    
    test_features = np.array(test_dataset.drop(['norm_value'], axis=1))
    test_features = test_features.reshape(test_features.shape[0],test_features.shape[1],1)
    test_features.shape

#     model = keras.Sequential()
#     model.add(layers.LSTM(128, return_sequences=True))
#     model.add(layers.TimeDistributed(layers.Dense(1)))
#     model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.005))
#     model.fit(train_features, train_labels, epochs=1, batch_size=1, verbose=2)
#     # load saved_model 
#     interpreter = tflite.Interpreter("./exported_models/tflite_model/LSTM_single_series.tflite")
#     input_details = interpreter.get_input_details()
#     output_details = interpreter.get_output_details()
#     # interpreter.resize_tensor_input(input_details[0]['index'],batch_input)
#     # interpreter.resize_tensor_input(output_details[0]['index'],batch_input)
#     input_details = interpreter.get_input_details()
#     output_details = interpreter.get_output_details()
#     interpreter.allocate_tensors()
#     interpreter.set_tensor(input_details[0]['index'], test_features)
#     interpreter.invoke()
#     old_model_result = interpreter.get_tensor(output_details[0]['index']) 


(24, 6, 1)

In [16]:
test_dataset.drop(['norm_6'], axis=1)

,norm_value,norm_1,norm_2,norm_3,norm_4,norm_5
7,0.587899,0.793949,0.793949,0.793949,0.587899,0.175797
11,0.381848,0.587899,0.587899,0.587899,0.587899,0.793949
12,0.072772,0.381848,0.587899,0.587899,0.587899,0.587899
15,-0.854456,-1.060507,-0.442355,0.072772,0.381848,0.587899
17,-0.030253,-0.648406,-0.854456,-1.060507,-0.442355,0.072772
18,-0.339330,-0.030253,-0.648406,-0.854456,-1.060507,-0.442355
20,0.587899,0.278823,-0.339330,-0.030253,-0.648406,-0.854456
22,0.793949,0.896975,0.587899,0.278823,-0.339330,-0.030253
24,0.587899,0.793949,0.793949,0.896975,0.587899,0.278823
26,-0.030253,0.175797,0.587899,0.793949,0.793949,0.896975
